# Importing necessary Libraries

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from gensim.models import Word2Vec, KeyedVectors
from gensim.utils import simple_preprocess
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api
from sklearn.model_selection import train_test_split


# Loading the csv files

In [2]:
books = pd.read_csv("Books.csv")
users = pd.read_csv("Users.csv")
ratings = pd.read_csv("Ratings.csv")

/var/folders/2j/xl2qzfzd781b4n_vgf720t2h0000gn/T/ipykernel_33168/2906776592.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("Books.csv")


In [3]:
books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [4]:
users

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [5]:
ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


# Data Cleaning and Preprocessing

In [6]:
df = pd.merge(ratings, books, on='ISBN', how='left')
df = df.drop(columns=["Image-URL-S", "Image-URL-M", "Image-URL-L"], axis=1)
df = df.dropna()
df['ISBN'] = df['ISBN'].astype(str)
df


,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle
2,276727,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books
3,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press
...,...,...,...,...,...,...,...
1149774,276704,0876044011,0,Edgar Cayce on the Akashic Records: The Book o...,Kevin J. Todeschi,1998,A.R.E. Press (Association of Research &amp; Enlig
1149775,276704,1563526298,9,Get Clark Smart : The Ultimate Guide for the S...,Clark Howard,2000,Longstreet Press
1149776,276706,0679447156,0,Eight Weeks to Optimum Health: A Proven Progra...,Andrew Weil,1997,Alfred A. Knopf
1149777,276709,0515107662,10,The Sherbrooke Bride (Bride Trilogy (Paperback)),Catherine Coulter,1996,Jove Books


Since the current dataset is too large for further matrix operations to be done on the system.The size of the dataset is reduced by random sampling, so that the overall distribution and characteristics of the dataset remains the same.  

In [7]:
df_sampled = df.sample(frac=0.1, random_state=42)  
df_sampled

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
357667,86189,038572179X,0,Atonement : A Novel,IAN MCEWAN,2003,Anchor
264821,61023,0345353145,8,Sphere,MICHAEL CRICHTON,1988,Ballantine Books
563445,135315,0553802380,10,The Next Accident,LISA GARDNER,2001,Bantam
94362,21031,078688908X,0,Chain of Evidence,Ridley Pearson,1997,Hyperion
52726,11676,0671776126,0,Plain Truth,Jodi Picoult,2000,Atria
...,...,...,...,...,...,...,...
633753,153662,0380768534,8,Avon Books Presents: Timeless Love,Lori Copeland,1993,Harper Mass Market Paperbacks (Mm)
303628,72575,0345441753,8,Criminal Intent,WILLIAM BERNHARDT,2003,Fawcett
148907,33580,1564146227,0,When Someone You Love Is Wiccan: A Guide to Wi...,Carl McColman,2002,New Page Books
876787,212583,0345367650,0,Polar Star,MARTIN CRUZ SMITH,1990,Ballantine Books


In [10]:
books_with_zero_rating = df_sampled[df_sampled['Book-Rating'] == 0]['Book-Title'].unique()
books_with_non_zero_ratings = df_sampled[df_sampled['Book-Rating'] > 0]['Book-Title'].unique()

print(f"Number of book titles with a rating of 0: {len(books_with_zero_rating)}")
print(f"Number of book titles with a rating above 0: {len(books_with_ratings_above_zero)}")

# print("Book titles with a rating of 0:", books_with_zero_rating)
# print("Book titles with a rating above 0:", books_with_ratings_above_zero)


Number of book titles with a rating of 0: 39260
Number of book titles with a rating above 0: 25340


In [11]:
df_sampled_final = df_sampled[df_sampled['Book-Title'].isin(books_with_non_zero_ratings)]
df_sampled_final


,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
357667,86189,038572179X,0,Atonement : A Novel,IAN MCEWAN,2003,Anchor
264821,61023,0345353145,8,Sphere,MICHAEL CRICHTON,1988,Ballantine Books
563445,135315,0553802380,10,The Next Accident,LISA GARDNER,2001,Bantam
94362,21031,078688908X,0,Chain of Evidence,Ridley Pearson,1997,Hyperion
52726,11676,0671776126,0,Plain Truth,Jodi Picoult,2000,Atria
...,...,...,...,...,...,...,...
770818,186570,0446358584,0,The Talbot Odyssey,Nelson DeMille,1991,Warner Books
837786,202688,0380715228,6,Pursuit of Happiness,David G. Myers,1993,Perennial Currents
633753,153662,0380768534,8,Avon Books Presents: Timeless Love,Lori Copeland,1993,Harper Mass Market Paperbacks (Mm)
303628,72575,0345441753,8,Criminal Intent,WILLIAM BERNHARDT,2003,Fawcett


# Creating the User-Item Rating Matrix

In [12]:
user_item_matrix = df_sampled_final.pivot_table(index='User-ID', columns='Book-Title', values='Book-Rating').fillna(0)

This matrix will have users as rows and books as columns, with ratings as the values. Books that a user hasn't rated will have a rating of 0.

# Collaborative Filtering

1. Calculate the similarity between users.
2. Predict ratings for books a user hasn't rated yet based on similar users' ratings.
3. Recommend books with the highest predicted ratings

In [13]:
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)


In [16]:
def recommend_books_based_on_book_title(book_title, user_item_matrix, user_similarity_df, top_n=10):
    if book_title not in user_item_matrix.columns:
        return "Book not found in our database."

    book_ratings = user_item_matrix[book_title]
    
    #print(f"Number of users who rated '{book_title}': {book_ratings[book_ratings > 0].count()}")

    users_who_rated_book = book_ratings[book_ratings > 0].index.tolist()

    if not users_who_rated_book:
        return "No users found who have rated this book."

    similar_users = user_similarity_df[users_who_rated_book].mean(axis=1)

    similar_users_sorted = similar_users.sort_values(ascending=False)

    weighted_ratings = pd.Series(dtype=float)

    for user_id, similarity in similar_users_sorted.iteritems():
        weighted_ratings = weighted_ratings.add(user_item_matrix.loc[user_id] * similarity, fill_value=0)

    weighted_ratings = weighted_ratings / similar_users_sorted.sum()

    recommended_books = weighted_ratings.sort_values(ascending=False)

    recommended_books = recommended_books.drop(book_title, errors='ignore')

    return recommended_books[recommended_books > 0].head(top_n)

# Get user input
user_book = input("Enter a book title: ").strip()

# Get recommendations
recommended_books = recommend_books_based_on_book_title(user_book, user_item_matrix, user_similarity_df)

if isinstance(recommended_books, str):
    print(recommended_books)
else:
    if recommended_books.empty:
        print("No recommendations found.")
    else:
        for book in recommended_books.index:
            print(book)


Enter a book title: Sphere


/var/folders/2j/xl2qzfzd781b4n_vgf720t2h0000gn/T/ipykernel_33168/3854482971.py:21: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for user_id, similarity in similar_users_sorted.iteritems():


Life of Pi
Love in the Time of Cholera (Penguin Great Books of the 20th Century)
Jitterbug Perfume
Red Dragon
Presumed Innocent
The Partner
Tell No One
Good in Bed
White Oleander : A Novel (Oprah's Book Club)
The Street Lawyer
